In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.resolve()))

from pprint import PrettyPrinter
pp = PrettyPrinter()

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
import openai
def wrap(func):
    def wrapper(*args, **kwargs):
        print("wrapping request=")
        pp.pprint(args)
        pp.pprint(kwargs)
        rets = func(*args, **kwargs)
        print("rets=")
        pp.pprint(rets)
        return rets
    return wrapper

for k in dir(openai):
    obj = getattr(openai, k)
    if hasattr(obj, "create"):
        print(f"wrapping {obj}")
        f = getattr(obj, "create")
        w = wrap(f)
        setattr(obj, "create", w)

wrapping <class 'openai.api_resources.chat_completion.ChatCompletion'>
wrapping <class 'openai.api_resources.completion.Completion'>
wrapping <class 'openai.api_resources.customer.Customer'>
wrapping <class 'openai.api_resources.deployment.Deployment'>
wrapping <class 'openai.api_resources.edit.Edit'>
wrapping <class 'openai.api_resources.embedding.Embedding'>
wrapping <class 'openai.api_resources.file.File'>
wrapping <class 'openai.api_resources.fine_tune.FineTune'>
wrapping <class 'openai.api_resources.image.Image'>
wrapping <class 'openai.api_resources.moderation.Moderation'>


In [3]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [4]:
from trulens_eval import feedback
provider_openai = feedback.OpenAI()
provider_openai.qs_relevance("Who is Piotr?", "Piotr is a person.")

openai api: 0requests [00:00, ?requests/s]

wrapping request=
()
{'messages': [{'content': 'You are a RELEVANCE classifier; providing the '
                          'relevance of the given STATEMENT to the given '
                          'QUESTION.\n'
                          'Respond only as a number from 1 to 10 where 1 is '
                          'the least relevant and 10 is the most relevant.\n'
                          'Never elaborate.\n'
                          '\n'
                          'QUESTION: Who is Piotr?\n'
                          '\n'
                          'STATEMENT: Piotr is a person.\n'
                          '\n'
                          'RELEVANCE: ',
               'role': 'system'}],
 'model': 'gpt-3.5-turbo',
 'temperature': 0.0}
rets=
{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'message': {'content': '10',
                          'role': 'assistant'}}],
 'created': 1686942304,
 'id': 'chatcmpl-7S92m7fGOpQGrBCgOkZ0kwelXfM5y',
 'model': 'gpt-3.5

1.0

In [ ]:
Tru().reset_database()

In [ ]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('llama_index/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [ ]:
# For aggregation,
import numpy as np

from trulens_eval import feedback, Feedback, Query, Tru

# Construct feedback functions.

hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [
#    f_lang_match, 
#    f_qa_relevance, 
#    f_qs_relevance
]

In [ ]:
l = Tru().Llama(engine=query_engine, feedbacks=feedbacks, chain_id="default")

In [ ]:
list(l.instrumented())

In [ ]:
res, record = l.query_with_record("Who is Shayak?")

In [ ]:
proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent)
# thread = Tru().start_evaluator(restart=True)

In [ ]:
import llama_index
dir(llama_index.llm_predictor)

In [ ]:
type(l.app._response_synthesizer._response_builder)